# Table of Content
1. [Introduction](#introduction)
2. [Preparation](#preparation)
3. [Feature Extraction](#feature_extraction)   
4. [EDA](#eda)
    1. [Time Series](#eda_time_series)
    2. [Missing Values (Professionals)](#eda_missing_values_professionals)
    3. [Missing Values (Students)](#eda_missing_values_students)
    4. [Tags Matching](#eda_tags_matching)
    5. [First Activity](#eda_first_activity)
    6. [Last Activity](#eda_last_activity)
    7. [First Answer](#eda_first_answer)
    8. [Email Response Time](#eda_email_response)
    9. [Word Count](#eda_wordcound)
5. [Ad Hoc Analysis](#ad_hoc)
    1. [User Activities](#ad_hoc_user_activites)
    2. [Dependency Graph](#ad_hoc_dependency)
6. [Recommendation](#recommendation)
    1. [Example 1](#recommender_example1)
    2. [Example 2](#recommender_example2)
7. [Topic Model (LDA)](#topic_model)
    1. [Model](#lda_model)
    2. [Topics](#lda_topics)
    3. [Document-Topic Probabilities](#lda_doc_topic_prob)
    4. [Example 1](#lda_example1)
    5. [Example 2](#lda_example2)

# 1. Introduction <a id="introduction"></a>  
With this notebook I would like to show some approaches for a new recommendation engine at [www.careervillage.org](http://www.careervillage.org).  
The figure below shows the data available to us and their dependencies. The most important datasets are "Questions", "Answers", "Professionals" and "Tags". Users can join a school oder group membership, which is currently not used by many.  
In Section [Preparation](#preparation) we load all necesary libraries, available csv files and set some global parameters. After this we use some [Feature extraction](#features_extraction) to create some new data columns for the analysis and natural language processing. We continue with some [Exploratory Data Analysis (EDA)](#eda), to get some insight into the current data.  
In addition, there is a section for [ad hoc analyses](#ad_hoc). These should make it possible to analyse the [activities of users](#ad_hoc_user_acitivies) and to check the [dependency](#ad_hoc_dependency) between questions, answers,  users and tags.  
With a [Topic Model (LDA)](#topic_model) we would like to identify different topics proportion of the questions. The advantage of LDA is that a question can have several topics and not only one. We can take advantage of this by identifying which topics are preferred by professionals when they answering the questions and use this for new recommendations.  
The [Recommendation Engine](#recommendation) itself should be answer the following questions (not yet finished):  
* When a student asks a question, are there already any similar questions that might help him?  
* Which professionals are most expected to answer the new question?  
* Which questions should be forwarded to a professional to answer?  
* Which hashtags might interest a user because of his previous activities?


![workflow_diagram](https://i.imgur.com/zzAo1JD.jpg)

# 2. Preparation <a id="preparation"></a>

## Libraries

In [ ]:
import os
#import datetime
import math
import random
import warnings

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.dates as mdates
from datetime import datetime
import networkx as nx

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import spacy
nlp = spacy.load('en')
nlp.remove_pipe('parser')
nlp.remove_pipe('ner')
#nlp.remove_pipe('tagger')

from wordcloud import WordCloud

import gensim
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

#import plotly.offline as py
#py.init_notebook_mode(connected=True)

## Read CSV Files

In [ ]:
input_dir = '../input'
#print(os.listdir(input_dir))

professionals = pd.read_csv(os.path.join(input_dir, 'professionals.csv'))
groups = pd.read_csv(os.path.join(input_dir, 'groups.csv'))
comments = pd.read_csv(os.path.join(input_dir, 'comments.csv'))
school_memberships = pd.read_csv(os.path.join(input_dir, 'school_memberships.csv'))
tags = pd.read_csv(os.path.join(input_dir, 'tags.csv'))
emails = pd.read_csv(os.path.join(input_dir, 'emails.csv'))
group_memberships = pd.read_csv(os.path.join(input_dir, 'group_memberships.csv'))
answers = pd.read_csv(os.path.join(input_dir, 'answers.csv'))
students = pd.read_csv(os.path.join(input_dir, 'students.csv'))
matches = pd.read_csv(os.path.join(input_dir, 'matches.csv'))
questions = pd.read_csv(os.path.join(input_dir, 'questions.csv'))
tag_users = pd.read_csv(os.path.join(input_dir, 'tag_users.csv'))
tag_questions = pd.read_csv(os.path.join(input_dir, 'tag_questions.csv'))

## Global Parameters

In [ ]:
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', -1)

seed = 13
random.seed(seed)
np.random.seed(seed)

# 3. Features extraction <a id="feature_extraction"></a> 

## Parameters

In [ ]:
# Spacy Tokenfilter for part-of-speech tagging
token_pos = ['NOUN', 'VERB', 'PROPN', 'ADJ', 'INTJ', 'X']

# The data export was from 1. February 2019. For Production use datetime.now()
actual_date = datetime(2019, 2 ,1)

## Functions

In [ ]:
def nlp_preprocessing(data):
    """ Use NLP to transform the text corpus to cleaned sentences and word tokens

    """    
    def token_filter(token):
        """ Keep tokens who are alphapetic, in the pos (part-of-speech) list and not in stop list

        """    
        return not token.is_stop and token.is_alpha and token.pos_ in token_pos
    
    processed_tokens = []
    data_pipe = nlp.pipe(data)
    for doc in data_pipe:
        filtered_tokens = [token.lemma_.lower() for token in doc if token_filter(token)]
        processed_tokens.append(filtered_tokens)
    return processed_tokens

## Features
First we transform the datetime columns (*date_added and date_joined*), so that we can work with time delta functions.  
After this we create the following new columns:  

**DataFrame Questions:**  
* questions_full_text: Merge the questions title with the body for later use of NLP.  
* questions_answers_count: How many answers a question has.  
* questions_first_answers: The timestamp for the first answer of the question.  
* questions_last_answers: The timestamp for the last answer of the question.  
* nlp_tokens: Extract relevant tokens from the question full text.  

**DataFrame Answers:**  
* time_delta_answert: Time delta from question to answer.  

**DataFrame Professionals:**  
* professionals_time_delta_joined: Time since creating the account.  
* professionals_answers_count: Number of written answers.  
* professionals_comments_count: Number of written comments.  
* date_last_answer: Date last answer.  
* date_first_answer: Date first answer.  
* date_last_comment: Date last comment.  
* date_first_comment: Date first comment.  
* date_last_activity: Date last activity (answer or comment).  
* date_first_activity: Date first activity (answer or comment).  

**DataFrame Students:**  
* students_time_delta_joined: Time since creating the account.  
* students_questions_count: Number of written questions.  
* students_comments_count: Number of written comments.  
* date_last_questions: Date last question.  
* date_first_questions: Date first question.  
* date_last_comment: Date last comment.  
* date_first_comment: Date first comment.  
* date_last_activity: Date last activity (question or comment).  
* date_first_activity: Date first activity (question or comment).  

**New DataFrame emails_response:**  
Has the response activity from professionals to emails and additional informations about the questions behind.
* time_delta_email_answer: Time needed the question was answered after the email was send.  
* time_delta_question_email: Time needed the email was send after the questions was written.

In [ ]:
%%time
# Transform datatypes
questions['questions_date_added'] = pd.to_datetime(questions['questions_date_added'], infer_datetime_format=True)
answers['answers_date_added'] = pd.to_datetime(answers['answers_date_added'], infer_datetime_format=True)
professionals['professionals_date_joined'] = pd.to_datetime(professionals['professionals_date_joined'], infer_datetime_format=True)
students['students_date_joined'] = pd.to_datetime(students['students_date_joined'], infer_datetime_format=True)
emails['emails_date_sent'] = pd.to_datetime(emails['emails_date_sent'], infer_datetime_format=True)
comments['comments_date_added'] = pd.to_datetime(comments['comments_date_added'], infer_datetime_format=True)

### Questions
# Merge Question Title and Body
questions['questions_full_text'] = questions['questions_title'] +'\r\n\r\n'+ questions['questions_body']
# Count of answers
temp = answers.groupby('answers_question_id').size()
questions['questions_answers_count'] = pd.merge(questions, pd.DataFrame(temp.rename('count')), left_on='questions_id', right_index=True, how='left')['count'].fillna(0).astype(int)
# First answer for questions
temp = answers[['answers_question_id', 'answers_date_added']].groupby('answers_question_id').min()
questions['questions_first_answers'] = pd.merge(questions, pd.DataFrame(temp), left_on='questions_id', right_index=True, how='left')['answers_date_added']
# Last answer for questions
temp = answers[['answers_question_id', 'answers_date_added']].groupby('answers_question_id').max()
questions['questions_last_answers'] = pd.merge(questions, pd.DataFrame(temp), left_on='questions_id', right_index=True, how='left')['answers_date_added']
# Get NLP Tokens
questions['nlp_tokens'] = nlp_preprocessing(questions['questions_full_text'])

### Answers
# Days required to answer the question
temp = pd.merge(questions, answers, left_on='questions_id', right_on='answers_question_id')
answers['time_delta_answer'] = (temp['answers_date_added'] - temp['questions_date_added'])


### Professionals
# Time since joining
professionals['professionals_time_delta_joined'] = actual_date - professionals['professionals_date_joined']
# Number of answers
temp = answers.groupby('answers_author_id').size()
professionals['professionals_answers_count'] = pd.merge(professionals, pd.DataFrame(temp.rename('count')), left_on='professionals_id', right_index=True, how='left')['count'].fillna(0).astype(int)
# Number of comments
temp = comments.groupby('comments_author_id').size()
professionals['professionals_comments_count'] = pd.merge(professionals, pd.DataFrame(temp.rename('count')), left_on='professionals_id', right_index=True, how='left')['count'].fillna(0).astype(int)
# Last activity (Answer)
temp = answers.groupby('answers_author_id')['answers_date_added'].max()
professionals['date_last_answer'] = pd.merge(professionals, pd.DataFrame(temp.rename('last_answer')), left_on='professionals_id', right_index=True, how='left')['last_answer']
# First activity (Answer)
temp = answers.groupby('answers_author_id')['answers_date_added'].min()
professionals['date_first_answer'] = pd.merge(professionals, pd.DataFrame(temp.rename('first_answer')), left_on='professionals_id', right_index=True, how='left')['first_answer']
# Last activity (Comment)
temp = comments.groupby('comments_author_id')['comments_date_added'].max()
professionals['date_last_comment'] = pd.merge(professionals, pd.DataFrame(temp.rename('last_comment')), left_on='professionals_id', right_index=True, how='left')['last_comment']
# First activity (Comment)
temp = comments.groupby('comments_author_id')['comments_date_added'].min()
professionals['date_first_comment'] = pd.merge(professionals, pd.DataFrame(temp.rename('first_comment')), left_on='professionals_id', right_index=True, how='left')['first_comment']
# Last activity (Total)
professionals['date_last_activity'] = professionals[['date_last_answer', 'date_last_comment']].max(axis=1)
# First activity (Total)
professionals['date_first_activity'] = professionals[['date_first_answer', 'date_first_comment']].min(axis=1)

### Students
# Time since joining
students['students_time_delta_joined'] = actual_date - students['students_date_joined']
# Number of answers
temp = questions.groupby('questions_author_id').size()
students['students_questions_count'] = pd.merge(students, pd.DataFrame(temp.rename('count')), left_on='students_id', right_index=True, how='left')['count'].fillna(0).astype(int)
# Number of comments
temp = comments.groupby('comments_author_id').size()
students['students_comments_count'] = pd.merge(students, pd.DataFrame(temp.rename('count')), left_on='students_id', right_index=True, how='left')['count'].fillna(0).astype(int)
# Last activity (Question)
temp = questions.groupby('questions_author_id')['questions_date_added'].max()
students['date_last_question'] = pd.merge(students, pd.DataFrame(temp.rename('last_question')), left_on='students_id', right_index=True, how='left')['last_question']
# First activity (Question)
temp = questions.groupby('questions_author_id')['questions_date_added'].min()
students['date_first_question'] = pd.merge(students, pd.DataFrame(temp.rename('first_question')), left_on='students_id', right_index=True, how='left')['first_question']
# Last activity (Comment)
temp = comments.groupby('comments_author_id')['comments_date_added'].max()
students['date_last_comment'] = pd.merge(students, pd.DataFrame(temp.rename('last_comment')), left_on='students_id', right_index=True, how='left')['last_comment']
# First activity (Comment)
temp = comments.groupby('comments_author_id')['comments_date_added'].min()
students['date_first_comment'] = pd.merge(students, pd.DataFrame(temp.rename('first_comment')), left_on='students_id', right_index=True, how='left')['first_comment']
# Last activity (Total)
students['date_last_activity'] = students[['date_last_question', 'date_last_comment']].max(axis=1)
# First activity (Total)
students['date_first_activity'] = students[['date_first_question', 'date_first_comment']].min(axis=1)

### Emails Response
emails_response = pd.merge(emails, matches, left_on='emails_id', right_on='matches_email_id', how='inner')
emails_response = pd.merge(emails_response, questions, left_on='matches_question_id', right_on='questions_id', how='inner')
emails_response = pd.merge(emails_response, answers, left_on=['emails_recipient_id', 'matches_question_id'], right_on=['answers_author_id', 'answers_question_id'], how='left')
emails_response = emails_response.drop(['matches_email_id', 'matches_question_id', 'answers_id', 'answers_author_id', 'answers_body', 'answers_question_id'], axis=1)
emails_response = emails_response.drop(['questions_author_id', 'questions_title', 'questions_body', 'questions_full_text'], axis=1)
emails_response['time_delta_email_answer'] = (emails_response['answers_date_added'] - emails_response['emails_date_sent'])
emails_response['time_delta_question_email'] = (emails_response['emails_date_sent'] - emails_response['questions_date_added'])

# 3. EDA <a id="eda"></a>

## Time Series <a id="eda_time_series"></a> 
Here we can see in which year the most user activity was. There was a large increase in 2016. 40% of all questions and comments were in this year.  
In some future analyses, we will limit the data to 2016 and onwards. This is to prevent possible noise from the Career Village start time.

In [ ]:
plt_questions = (questions.groupby([questions['questions_date_added'].dt.year]).size()/len(questions.index))
plt_answers = (answers.groupby([answers['answers_date_added'].dt.year]).size()/len(answers.index))
plt_emails = (emails.groupby([emails['emails_date_sent'].dt.year]).size()/len(emails.index))
plt_comments = (comments.groupby([comments['comments_date_added'].dt.year]).size()/len(comments.index))
plt_data = pd.DataFrame({'questions': plt_questions,
                        'answers':plt_answers,
                        'emails':plt_emails,
                        'comments':plt_comments})
plt_data.plot(kind='bar', figsize=(15, 5))
plt.xlabel('Year')
plt.ylabel('Proportion')
plt.title('Distribution over time')
plt.show()

## Missing values (Professionals) <a id="eda_missing_values_professionals"></a> 
The *Location, Industry and Headline* were specified by the most professionals. Even hashtags are used by most. The specification of the school is made however only by the fewest.  
The industry could therefore be a good feature for the recommendation. Especially for new authors who have not written any answer or comments. If an professional is in the medical field, he should not necessarily get questions about a career as a lawyer.


In [ ]:
temp = professionals[['professionals_location', 'professionals_industry', 'professionals_headline']].fillna('Missing')
temp = temp.applymap(lambda x: x if x == 'Missing' else 'Available')
plt_professionals_location = temp.groupby('professionals_location').size()/len(temp.index)
plt_professionals_industry = temp.groupby('professionals_industry').size()/len(temp.index)
plt_professionals_headline = temp.groupby('professionals_headline').size()/len(temp.index)
plt_professionals_tags = tag_users['tag_users_user_id'].unique()
plt_professionals_tags = professionals['professionals_id'].apply(lambda x: 'Available' if x in plt_professionals_tags else 'Missing').rename('professionals_tags')
plt_professionals_tags = plt_professionals_tags.groupby(plt_professionals_tags).size()/len(plt_professionals_tags.index)
plt_professionals_group = group_memberships['group_memberships_user_id'].unique()
plt_professionals_group = professionals['professionals_id'].apply(lambda x: 'Available' if x in plt_professionals_group else 'Missing').rename('professionals_groups')
plt_professionals_group = plt_professionals_group.groupby(plt_professionals_group).size()/len(plt_professionals_group.index)
plt_professionals_school = school_memberships['school_memberships_user_id'].unique()
plt_professionals_school = professionals['professionals_id'].apply(lambda x: 'Available' if x in plt_professionals_school else 'Missing').rename('professionals_schools')
plt_professionals_school = plt_professionals_school.groupby(plt_professionals_school).size()/len(plt_professionals_school.index)
temp = professionals[['professionals_answers_count', 'professionals_comments_count']]
temp = temp.applymap(lambda x: 'Available' if x > 0 else 'Missing')
plt_professionals_answers = temp.groupby('professionals_answers_count').size()/len(temp.index)
plt_professionals_comments = temp.groupby('professionals_comments_count').size()/len(temp.index)

plt_data = pd.DataFrame({'Location': plt_professionals_location,
                        'Industry': plt_professionals_industry,
                        'Headline': plt_professionals_headline,
                        'Tags': plt_professionals_tags,
                        'Groups': plt_professionals_group,
                        'Schools': plt_professionals_school,
                        'Answers': plt_professionals_answers,
                        'Comments': plt_professionals_comments,})

plt_data.T.plot(kind='bar', stacked=True, figsize=(15, 5))
plt.ylabel('Proportion')
plt.title('Missing values for professionals')
plt.yticks(np.arange(0, 1.05, 0.1))
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()
plt_data.T

## Missing values (Students) <a id="eda_missing_values_students"></a> 
It's a little different with the students. Only the location is specified by most students, while the rest is rather not used.

In [ ]:
temp = students[['students_location']].fillna('Missing')
temp = temp.applymap(lambda x: x if x == 'Missing' else 'Available')
plt_students_location = temp.groupby('students_location').size()/len(temp.index)
plt_students_tags = tag_users['tag_users_user_id'].unique()
plt_students_tags = students['students_id'].apply(lambda x: 'Available' if x in plt_students_tags else 'Missing').rename('students_tags')
plt_students_tags = plt_students_tags.groupby(plt_students_tags).size()/len(plt_students_tags.index)
plt_students_group = group_memberships['group_memberships_user_id'].unique()
plt_students_group = students['students_id'].apply(lambda x: 'Available' if x in plt_students_group else 'Missing').rename('students_groups')
plt_students_group = plt_students_group.groupby(plt_students_group).size()/len(plt_students_group.index)
plt_students_school = school_memberships['school_memberships_user_id'].unique()
plt_students_school = students['students_id'].apply(lambda x: 'Available' if x in plt_students_school else 'Missing').rename('students_schools')
plt_students_school = plt_students_school.groupby(plt_students_school).size()/len(plt_students_school.index)
temp = students[['students_questions_count', 'students_comments_count']]
temp = temp.applymap(lambda x: 'Available' if x > 0 else 'Missing')
plt_students_questions = temp.groupby('students_questions_count').size()/len(temp.index)
plt_students_comments = temp.groupby('students_comments_count').size()/len(temp.index)

plt_data = pd.DataFrame({'Location': plt_students_location,
                        'Tags': plt_students_tags,
                        'Groups': plt_students_group,
                        'Schools': plt_students_school,
                        'Answers': plt_students_questions,
                        'Comments': plt_students_comments,})

plt_data.T.plot(kind='bar', stacked=True, figsize=(15, 5))
plt.ylabel('Proportion')
plt.title('Missing values for students')
plt.yticks(np.arange(0, 1.05, 0.1))
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()
plt_data.T

## Tags matching <a id="eda_tags_matching"></a> 
The size of the bubbles depends on how many questions the tag is used. The x-axis is how many professionals have subscribe the tag and the y-axis is how many students have subscribe the tag.  
The top tag for professionals ist *telecommunications* on the right site with about 11% but the tag doesn't appear in many questions or students subscribtion.  
The top tags for questions is *college* with 15.6% and *carrer* with 6.5%. The other top tags are carrer specific (*medicine, engineering, business, ...*).  
The top tag for students is *college* but only 1.5% of the students have subscribe this tag.  

In [ ]:
students_tags = tag_users[tag_users['tag_users_user_id'].isin(students['students_id'])]
students_tags = pd.merge(students_tags, tags, left_on='tag_users_tag_id', right_on='tags_tag_id')
students_tags['user_type'] = 'student'
professionals_tags = tag_users[tag_users['tag_users_user_id'].isin(professionals['professionals_id'])]
professionals_tags = pd.merge(professionals_tags, tags, left_on='tag_users_tag_id', right_on='tags_tag_id')
professionals_tags['user_type'] = 'professional'
questions_tags = tag_questions
questions_tags = pd.merge(questions_tags, tags, left_on='tag_questions_tag_id', right_on='tags_tag_id')
questions_tags['user_type'] = 'question'
plt_data = pd.concat([students_tags, professionals_tags, questions_tags])

plt_data = plt_data[['tags_tag_name', 'user_type']].pivot_table(index='tags_tag_name', columns='user_type', aggfunc=len, fill_value=0)
plt_data['professional'] = plt_data['professional'] / professionals.shape[0]
plt_data['student'] = plt_data['student'] / students.shape[0]
plt_data['question'] = plt_data['question'] / questions.shape[0]
plt_data['sum'] = (plt_data['professional'] + plt_data['student'] + plt_data['question'])
plt_data = plt_data.sort_values(by='sum', ascending=False).drop(['sum'], axis=1).head(100)

# Bubble chart
fig, ax = plt.subplots(facecolor='w',figsize=(15, 15))
ax.set_xlabel('Professionals')
ax.set_ylabel('Students')
ax.set_title('Tags Matching')
ax.set_xlim([0, max(plt_data['professional'])+0.001])
ax.set_ylim([0, max(plt_data['student'])+0.001])
import matplotlib.ticker as mtick
ax.xaxis.set_major_formatter(mtick.FuncFormatter("{:.2%}".format))
ax.yaxis.set_major_formatter(mtick.FuncFormatter("{:.2%}".format))
ax.grid(True)
i = 0
for key, row in plt_data.iterrows():
    ax.scatter(row['professional'], row['student'], s=row['question']*10**4, alpha=.5)
    if i < 25:
        ax.annotate('{}: {:.2%}'.format(key, row['question']), xy=(row['professional'], row['student']))
    i += 1
plt.show()

# Wordcloud
plt.figure(figsize=(20, 20))
wordloud_values = ['student', 'professional', 'question']
axisNum = 1
for wordcloud_value in wordloud_values:
    wordcloud = WordCloud(margin=0, max_words=20, random_state=seed).generate_from_frequencies(plt_data[wordcloud_value])
    ax = plt.subplot(1, 3, axisNum)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(wordcloud_value)
    plt.axis("off")
    axisNum += 1
plt.show()    

## First activity after registration  <a id="eda_first_activity"></a>
Here we can see how long it took, that an professional makes his first answer or a student his first question after the registration.  
The most of them write the first answer/question within in the first day or haven't write any yet and use the account for other activities.

In [ ]:
plt_professionals = professionals
plt_professionals = plt_professionals[(plt_professionals['professionals_date_joined'] >= '01-01-2016') & (plt_professionals['professionals_date_joined'] <= '30-06-2018')]
plt_professionals = (plt_professionals['date_first_activity'] - plt_professionals['professionals_date_joined']).dt.days.fillna(9999).astype(int)
plt_professionals = plt_professionals.groupby(plt_professionals).size()/len(plt_professionals.index)
plt_professionals = plt_professionals.rename(lambda x: 0 if x < 0.0 else x)
plt_professionals = plt_professionals.rename(lambda x: x if x <= 7.0 or x == 9999 else '> 7')
plt_professionals = plt_professionals.rename({9999:'NaN'})
plt_professionals = plt_professionals.groupby(level=0).sum()

plt_students = students
plt_students = plt_students[(plt_students['students_date_joined'] >= '01-01-2016') & (plt_students['students_date_joined'] <= '30-06-2018')]
plt_students = (plt_students['date_first_activity'] - plt_students['students_date_joined']).dt.days.fillna(9999).astype(int)
plt_students = plt_students.groupby(plt_students).size()/len(plt_students.index)
plt_students = plt_students.rename(lambda x: 0 if x < 0.0 else x)
plt_students = plt_students.rename(lambda x: x if x <= 7.0 or x == 9999 else '> 7')
plt_students = plt_students.rename({9999:'NaN'})
plt_students = plt_students.groupby(level=0).sum()

plt_data = pd.DataFrame({'Professionals': plt_professionals,
                        'Students': plt_students})

plt_data.plot(kind='bar', figsize=(15, 5))
plt.xlabel('Days')
plt.ylabel('Proportion')
plt.title('Days for first activity after registration')
plt.show()

## Last activity  <a id="eda_last_activity"></a>  
Depending on the last comment, question or answer of a user, we have extract the last activity date. On the previously plot we have seen, that many users haven't done any activity yet. For the 'last activity' plot we take a look only on users with already have one activity (*dropna*).  
On the cumulative histogram we can see, that in the last 12 months only 39% of professionals and 24% of students have written a comment, question or answer.  
50% of the professionals haven't done any activity for 17 months.

In [ ]:
plt_professionals = ((actual_date - professionals['date_last_activity']).dt.days/30).dropna().apply(lambda x: math.ceil(x)).astype(int)
plt_professionals = plt_professionals.groupby(plt_professionals).size()/len(plt_professionals.index)
plt_professionals = plt_professionals.rename(lambda x: 0 if x < 0.0 else x)
plt_professionals = plt_professionals.rename(lambda x: x if x <= 36.0 or x == 9999 else '> 36')
plt_professionals = plt_professionals.rename({9999:'NaN'})
plt_professionals = plt_professionals.groupby(level=0).sum()

plt_students = ((actual_date - students['date_last_activity']).dt.days/30).dropna().apply(lambda x: math.ceil(x)).astype(int)
plt_students = plt_students.groupby(plt_students).size()/len(plt_students.index)
plt_students = plt_students.rename(lambda x: 0 if x < 0.0 else x)
plt_students = plt_students.rename(lambda x: x if x <= 36.0 or x == 9999 else '> 36')
plt_students = plt_students.rename({9999:'NaN'})
plt_students = plt_students.groupby(level=0).sum()

plt_data = pd.DataFrame({'Professionals': plt_professionals,
                        'Students': plt_students})

plt_data.plot(kind='bar', figsize=(15, 5))
plt.xlabel('Months')
plt.ylabel('Proportion')
plt.title('Months for last activity')
plt.show()

In [ ]:
plt_professionals = ((actual_date - professionals['date_last_activity']).dt.days/30).dropna().apply(lambda x: math.ceil(x)).astype(int)
plt_students = ((actual_date - students['date_last_activity']).dt.days/30).dropna().apply(lambda x: math.ceil(x)).astype(int)
plt_data = pd.DataFrame({'Professionals': plt_professionals,
                        'Students': plt_students})
plt_total = pd.concat([plt_data['Professionals'], plt_data['Students']]).rename('All Users')
plt_data.plot(kind='hist', bins=1000, density=True, histtype='step', cumulative=True, figsize=(15, 7), lw=2, grid=True)
plt_total.plot(kind='hist', bins=1000, density=True, histtype='step', cumulative=True, figsize=(15, 7), lw=2, grid=True)
plt.xlabel('Months')
plt.ylabel('Cumulative')
plt.title('Cumulative histogram for last activity')
plt.legend(loc='upper left')
plt.xlim([0, 50])
plt.xticks(range(0, 51, 1))
plt.yticks(np.arange(0, 1.05, 0.05))
plt.show()

## First answer for questions  <a id="eda_first_answer"></a>
Here we can see how long it takes for a question to get the first answer.  
The most questions get answered within the first two weeks. But 15% need more than 30 weeks to get there answer. And there are still some unanswered questions.  
The questions are filtered until 30-06-2018 to ignore new unanswered questions.

In [ ]:
plt_questions = questions
plt_questions = plt_questions[(plt_questions['questions_date_added'] >= '01-01-2016') & (plt_questions['questions_date_added'] <= '30-06-2018')]
plt_questions = ((plt_questions['questions_first_answers'] - plt_questions['questions_date_added']).dt.days/7).fillna(9999).apply(lambda x: math.ceil(x)).astype(int)
plt_questions = plt_questions.groupby(plt_questions).size()/len(plt_questions.index)
plt_questions = plt_questions.rename(lambda x: 0 if x < 0.0 else x)
plt_questions = plt_questions.rename(lambda x: x if x <= 30.0 or x == 9999 else '> 30')
plt_questions = plt_questions.rename({9999:'NaN'})
plt_questions = plt_questions.groupby(level=0).sum()

plt_data = pd.DataFrame({'Questions': plt_questions})

plt_data.plot(kind='bar', figsize=(15, 5))
plt.xlabel('Weeks')
plt.ylabel('Frequency')
plt.title('Weeks for first answer')
plt.show()

## Email response time  <a id="eda_email_response"></a>  
The first plot shows how long it takes for professionals to answer a question after they receive an email notification of a recommended question. This is limited to emails where the specified question was also answered (*dropna*).  When users want to answer a question from an email, they usually use the most recent email. The response time is within the first few days.  

On the second plot with the response time for questions, we can see the emails focusing on older unanswered questions. the immediate emails are focusing on new questions (makes sense).

In [ ]:
plt_email_response = emails_response[(emails_response['emails_date_sent'] >= '01-01-2016')].dropna()

plt_data = pd.DataFrame()
title_mapping = {'time_delta_email_answer':'Answers', 'time_delta_question_email':'Questions'}
for qa in ['time_delta_email_answer', 'time_delta_question_email']:
    plt_data = pd.DataFrame()
    for fl in ['email_notification_daily', 'email_notification_weekly', 'email_notification_immediate']:
        temp = plt_email_response[plt_email_response['emails_frequency_level'] == fl]
        temp = temp[qa].dt.days.astype(int)
        temp = temp.groupby(temp).size()/len(temp.index)
        temp = temp.rename(lambda x: 0 if x < 0.0 else x)
        temp = temp.rename(lambda x: x if x <= 14.0 else '> 14')
        temp = temp.groupby(level=0).sum() 
        plt_data = pd.concat([plt_data, temp], axis=1, sort=False)
    plt_data.columns = ['Daily', 'Weekly', 'Immediate']

    plt_data.plot(kind='bar', figsize=(15, 5))
    plt.xlabel('Days')
    plt.ylabel('Frequency')
    plt.title('Email response time ({})'.format(title_mapping[qa]))
    plt.legend(loc='upper center')
    plt.show()

## Word count <a id="eda_wordcound"></a>
Here we can see how many words are used for the questions and answers.  
The professionals write very detailed answers for the students questions.

In [ ]:
plt_data_questions = questions['questions_full_text'].apply(lambda x: len(x.split())).rename("Questions")
plt_data_answers = answers['answers_body'].astype(str).apply(lambda x: len(x.split())).rename("Answers")
plt_data = pd.DataFrame([plt_data_questions, plt_data_answers]).T
print(plt_data.describe())
plt_data.plot(kind='box', showfliers=False, vert=False, figsize=(15, 5), grid=True)
plt.xticks(range(0, 400, 25))
plt.xlabel('Words')
plt.title('Word count')
plt.show()

# 5. Ad Hoc Analysis <a id="ad_hoc"></a>  

## User Activities <a id="ad_hoc_user_activities"></a>  

In [ ]:
def plot_user_activity(user_id=seed, xticks_type='Month', xticks_interval=3):
    """ Merges all relevant data for a user together and builds a timeline chart.
        
        :param user_id: Index of the 'students' dataframe (default: seed)
        :param xticks_type: Grouping x axis by 'Month', 'Day' or 'Year' (default: 'Month')
        :param xticks_interval: Integer to plot every n ticks (default: 3)
    """  
    graph_student = students[students['students_id'] == user_id]
    graph_student = pd.DataFrame({'type':'Registration', 
                               'id':graph_student['students_id'], 
                               'date':graph_student['students_date_joined'],
                              'color':'green'})
    graph_professional = professionals[professionals['professionals_id'] == user_id]
    graph_professional = pd.DataFrame({'type':'Registration', 
                               'id':graph_professional['professionals_id'], 
                               'date':graph_professional['professionals_date_joined'],
                              'color':'green'})
    graph_questions = questions[questions['questions_author_id'] == user_id]
    graph_questions = pd.DataFrame({'type':'Questions',
                                   'id':graph_questions['questions_id'], 
                                   'date':graph_questions['questions_date_added'],
                                  'color':'blue'})
    graph_answers = answers[answers['answers_author_id'] == user_id]
    graph_answers = pd.DataFrame({'type':'Answers',
                                   'id':graph_answers['answers_id'], 
                                   'date':graph_answers['answers_date_added'],
                                  'color':'red'})
    graph_comments = comments[comments['comments_author_id'] == user_id]
    graph_comments = pd.DataFrame({'type':'Comments',
                                   'id':graph_comments['comments_id'], 
                                   'date':graph_comments['comments_date_added'],
                                  'color':'orange'})

    graph_data = pd.concat([graph_student, graph_professional, graph_questions, graph_comments, graph_answers])
    # Group data by date
    if xticks_type=='Day':
        graph_data['date'] = graph_data['date'].dt.strftime('%Y-%m-%d')
        graph_data['date'] = graph_data['date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
    else:
        graph_data['date'] = graph_data['date'].dt.strftime('%B %Y')
        graph_data['date'] = graph_data['date'].apply(lambda x: datetime.strptime(x, "%B %Y"))     
    
    graph_data = graph_data.groupby(['type', 'date', 'color']).size().rename('count').reset_index().sort_values('date')
    graph_data['name'] = graph_data['count'].map(str)+ ' '+graph_data['type']
    names = graph_data['name'].tolist()
    colors = graph_data['color'].tolist()
    dates = graph_data['date'].tolist()

    # Plot
    levels = np.array([-7, 7, -5, 5, -3, 3, -1, 1])
    fig, ax = plt.subplots(figsize=(15, 8))

    # Create the base line
    start = min(dates)
    stop = max(dates)
    ax.plot((start, stop), (0, 0), 'k', alpha=.5)

    # Create annotations
    for ii, (iname, idate, icol) in enumerate(zip(names, dates, colors)):
        level = levels[ii % len(levels)]
        vert = 'top' if level < 0 else 'bottom'
        ax.scatter(idate, 0, s=100, facecolor=icol, edgecolor='k', zorder=9999)
        ax.plot((idate, idate), (0, level), c=icol, alpha=1.0, lw=2)
        ax.text(idate, level, iname, horizontalalignment='center', verticalalignment=vert, fontsize=12, backgroundcolor=icol)
    ax.set(title="Timeline for user: {}".format(user_id))
    # Set the xticks formatting
    if xticks_type=='Month':
        ax.get_xaxis().set_major_locator(mdates.MonthLocator(interval=xticks_interval))
        ax.get_xaxis().set_major_formatter(mdates.DateFormatter("%B %Y"))
    elif xticks_type=='Day':
        ax.get_xaxis().set_major_locator(mdates.DayLocator(interval=xticks_interval))
        ax.get_xaxis().set_major_formatter(mdates.DateFormatter("%Y-%m-%d"))
    elif xticks_type=='Year':
        ax.get_xaxis().set_major_locator(mdates.YearLocator())
        ax.get_xaxis().set_major_formatter(mdates.DateFormatter("%Y"))        
    fig.autofmt_xdate()
    #Legend
    legend = []
    for index, row in graph_data[['type', 'color']].drop_duplicates().iterrows():
        legend += [mpatches.Patch(color=row['color'], label=row['type'])]
    plt.legend(handles=legend, loc='center left', bbox_to_anchor=(1, 0.5))
    
    # Remove components for a cleaner look
    plt.setp((ax.get_yticklabels() + ax.get_yticklines() + list(ax.spines.values())), visible=False)
    plt.show()

## Example 1 (Professionals)

In [ ]:
user_id = '977428d851b24183b223be0eb8619a8c'
plot_user_activity(user_id=user_id, xticks_type='Month', xticks_interval=1)
user_id = 'e1d39b665987455fbcfbec3fc6df6056'
plot_user_activity(user_id=user_id, xticks_type='Month', xticks_interval=3)

## Example 2 (Students)

In [ ]:
user_id = '16908136951a48ed942738822cedd5c2'
plot_user_activity(user_id=user_id, xticks_type='Month', xticks_interval=1)
user_id = 'e5c389a88c884e13ac828dd22628acc8'
plot_user_activity(user_id=user_id, xticks_type='Day', xticks_interval=7)

## Dependency Graph <a id="ad_hoc_dependency"></a>  

In [ ]:
def plot_dependecy_graph(emails_id=[], details=['tag', 'group', 'school'], details_min_edges=1):
    """ Merges all relevant data for a given email together and builds a dependency graph and report.
        
        :param emails_id: 'email_id' of the 'emails' dataframe
        :param details: List which details should be ploted (default: ['tags', 'groups', 'schools'])
    """  
    graph_edges = pd.DataFrame()
    graph_nodes = pd.DataFrame()
    #email
    graph_emails = emails[emails['emails_id'].isin(emails_id)]
    graph_nodes = pd.concat([graph_nodes, pd.DataFrame({'node':graph_emails['emails_id'], 'type':'email', 'color':'grey', 'size':1})])
    #questions
    graph_matches = matches[matches['matches_email_id'].isin(graph_emails['emails_id'])]
    graph_nodes = pd.concat([graph_nodes, pd.DataFrame({'node':graph_matches['matches_question_id'], 'type':'question', 'color':'blue', 'size':1})])
    graph_edges = pd.concat([graph_edges, pd.DataFrame({'source':graph_matches['matches_email_id'], 'target':graph_matches['matches_question_id']})])
    #answers
    graph_answers = answers[answers['answers_question_id'].isin(graph_matches['matches_question_id'])]
    graph_nodes = pd.concat([graph_nodes, pd.DataFrame({'node':graph_answers['answers_id'], 'type':'answer', 'color':'red', 'size':1})])
    graph_edges = pd.concat([graph_edges, pd.DataFrame({'source':graph_answers['answers_question_id'], 'target':graph_answers['answers_id']})])
    #professionals
    graph_professionals = answers[answers['answers_question_id'].isin(graph_matches['matches_question_id'])]
    graph_nodes = pd.concat([graph_nodes, pd.DataFrame({'node':graph_professionals['answers_author_id'], 'type':'professional', 'color':'cyan', 'size':1})])
    graph_edges = pd.concat([graph_edges, pd.DataFrame({'source':graph_professionals['answers_id'], 'target':graph_professionals['answers_author_id']})])
    #students
    graph_students = questions[questions['questions_id'].isin(graph_matches['matches_question_id'])]
    graph_nodes = pd.concat([graph_nodes, pd.DataFrame({'node':graph_students['questions_author_id'], 'type':'student', 'color':'green', 'size':1})])
    graph_edges = pd.concat([graph_edges, pd.DataFrame({'source':graph_students['questions_id'], 'target':graph_students['questions_author_id']})])
    if 'tag' in details:
        #question tags
        graph_questions_tags = tag_questions[tag_questions['tag_questions_question_id'].isin(graph_matches['matches_question_id'])]
        graph_questions_tags = pd.merge(graph_questions_tags, tags, left_on='tag_questions_tag_id', right_on='tags_tag_id')
        graph_nodes = pd.concat([graph_nodes, pd.DataFrame({'node':graph_questions_tags['tags_tag_name'], 'type':'tag', 'color':'yellow', 'size':1})])
        graph_edges = pd.concat([graph_edges, pd.DataFrame({'source':graph_questions_tags['tag_questions_question_id'], 'target':graph_questions_tags['tags_tag_name']})])  
        #professional tags
        graph_professionals_tags = tag_users[tag_users['tag_users_user_id'].isin(graph_professionals['answers_author_id'])]
        graph_professionals_tags = pd.merge(graph_professionals_tags, tags, left_on='tag_users_tag_id', right_on='tags_tag_id')
        graph_nodes = pd.concat([graph_nodes, pd.DataFrame({'node':graph_professionals_tags['tags_tag_name'], 'type':'tag', 'color':'yellow', 'size':1})])
        graph_edges = pd.concat([graph_edges, pd.DataFrame({'source':graph_professionals_tags['tag_users_user_id'], 'target':graph_professionals_tags['tags_tag_name']})])     
        #students tags
        graph_students_tags = tag_users[tag_users['tag_users_user_id'].isin(graph_students['questions_author_id'])]
        graph_students_tags = pd.merge(graph_students_tags, tags, left_on='tag_users_tag_id', right_on='tags_tag_id')
        graph_nodes = pd.concat([graph_nodes, pd.DataFrame({'node':graph_students_tags['tags_tag_name'], 'type':'tag', 'color':'yellow', 'size':1})])
        graph_edges = pd.concat([graph_edges, pd.DataFrame({'source':graph_students_tags['tag_users_user_id'], 'target':graph_students_tags['tags_tag_name']})]) 
    if 'group' in details:
        #professional group
        graph_professionals_group = group_memberships[group_memberships['group_memberships_user_id'].isin(graph_professionals['answers_author_id'])]
        graph_nodes = pd.concat([graph_nodes, pd.DataFrame({'node':graph_professionals_group['group_memberships_group_id'], 'type':'group', 'color':'orange', 'size':1})])
        graph_edges = pd.concat([graph_edges, pd.DataFrame({'source':graph_professionals_group['group_memberships_user_id'], 'target':graph_professionals_group['group_memberships_group_id']})]) 
        #students group
        graph_students_group = group_memberships[group_memberships['group_memberships_user_id'].isin(graph_students['questions_author_id'])]
        graph_nodes = pd.concat([graph_nodes, pd.DataFrame({'node':graph_students_group['group_memberships_group_id'], 'type':'group', 'color':'orange', 'size':1})])
        graph_edges = pd.concat([graph_edges, pd.DataFrame({'source':graph_students_group['group_memberships_user_id'], 'target':graph_students_group['group_memberships_group_id']})]) 
    if 'school' in details:
        #professional school
        graph_professionals_school = school_memberships[school_memberships['school_memberships_user_id'].isin(graph_professionals['answers_author_id'])]
        graph_nodes = pd.concat([graph_nodes, pd.DataFrame({'node':graph_professionals_school['school_memberships_school_id'], 'type':'school', 'color':'purple', 'size':1})])
        graph_edges = pd.concat([graph_edges, pd.DataFrame({'source':graph_professionals_school['school_memberships_user_id'], 'target':graph_professionals_school['school_memberships_school_id']})])
        #students school
        graph_students_school = school_memberships[school_memberships['school_memberships_user_id'].isin(graph_students['questions_author_id'])]
        graph_nodes = pd.concat([graph_nodes, pd.DataFrame({'node':graph_students_school['school_memberships_school_id'], 'type':'school', 'color':'purple', 'size':1})])
        graph_edges = pd.concat([graph_edges, pd.DataFrame({'source':graph_students_school['school_memberships_user_id'], 'target':graph_students_school['school_memberships_school_id']})])
    
    # check min count of edges for details
    graph_nodes = graph_nodes.drop_duplicates()
    temp = pd.concat([graph_edges[['source', 'target']], graph_edges[['target', 'source']].rename(columns={'target':'source', 'source':'target'})])
    temp = temp[temp['source'].isin(graph_nodes[graph_nodes['type'].isin(details)]['node'])]
    temp = temp.drop_duplicates().groupby('source').size()
    graph_nodes = graph_nodes[~graph_nodes['node'].isin(temp[temp<details_min_edges].index.values)]
    graph_edges = graph_edges[(~graph_edges['source'].isin(temp[temp<details_min_edges].index.values)) & (~graph_edges['target'].isin(temp[temp<details_min_edges].index.values))]
    graph_nodes_color = graph_nodes['color']
    
    plt.figure(figsize=(15, 15)) 
    G = nx.Graph()
    G.add_nodes_from(graph_nodes['node'])
    G.add_edges_from({tuple(row) for i,row in graph_edges[['source', 'target']].iterrows()})
    nx.draw_networkx(G, with_labels=True, node_color=graph_nodes_color, font_size=8, node_size=900/len(emails_id))
    plt.title('Dependency graph for email {}'.format(emails_id))
    plt.axis('off')

    legend = []
    for index, row in graph_nodes[['type', 'color']].drop_duplicates().iterrows():
        legend += [mpatches.Patch(color=row['color'], label=row['type'])]
    plt.legend(handles=legend)
    plt.show()

## Example 1  <a id="dependency_graph_example1"></a> 
In example 1 we have <span style="background-color:gray">one email</span> with <span style="background-color:blue">two questions</span> with a few <span style="background-color:yellow">questions tags</span>. There are <span style="background-color:red">several answers</span> for the question. The <span style="background-color:green">students</span> from the questions haven't any <span style="background-color:orange">group</span> or <span style="background-color:purple">school</span> membership. The <span style="background-color:cyan">professionals</span> have more motivation to subscribe some <span style="background-color:yellow">tags</span> and specify there <span style="background-color:purple">school</span> membership. But only <span style="background-color:cyan">two professionals</span> have joined a <span style="background-color:orange">group</span>.<span style="background-color:blue"></span>  

In the second plot of this example we increase the parameter *details_min_edges* to 2. This provides a better overview of which tags are used by multiple nodes. For example, the 'financial-planing' node is used in two questions and is subscribed by the professional and student at the same time.

In [ ]:
emails_id = emails.loc[seed, 'emails_id']
plot_dependecy_graph(emails_id=[emails_id])

In [ ]:
emails_id = emails.loc[seed, 'emails_id']
plot_dependecy_graph(emails_id=[emails_id], details_min_edges=2)

## Example 1  <a id="dependency_graph_example2"></a> 

In [ ]:
emails_id = emails.loc[seed*2, 'emails_id']
plot_dependecy_graph(emails_id=[emails_id])

# 5. Recommendation <a id="recommendation"></a>  
With the preprocessed data I build a tf-idf corpus and can use this, to calculate the (cosine) similarity between a new question and the given questions.  
Here are the detailed steps:  
1. Use NLP on the Questions corpus.  
    a. Use part-of-speech tagging to filter words.  
    b. Calculate the tf-idf for a better Information Retrieval.  
2. Use NLP on the Query text.  
    a. Use part-of-speech tagging to filter words.  
    b. Calculate the tf-idf for a better Information Retrieval.  
3. Use the cosine similiarty to get similiar questions for the query text.  
4. Get the answers and professionals for the similar questions.  
5. Make a recommendation to fit the best professionals to answer the new question.  

I use the similar questions and the professionals who answered the question to calculate a *recommendation score*. On the basis of this, professionals will be recommended who have already answered many similar questions which have not been answered by many others. The first draft of this formula is as follows  
$Professional_{score} = \sum\limits_{q}^{Q}(q_{sim}*\dfrac{1}{q_{answers}}*p_{answers})$  
$Q$ = Similar  questions  answered  by  professional  p  
$q_{sim}$ = Similarity of the question q  
$q_{answers}$ = Total answers for question q  
$p_{answers}$ = Total answers of professional p
    

## Functions

In [ ]:
def get_similar_docs(corpus, query_text, threshold=0.0, top=5):
    """ Calculates the tfidf of the corpus and returns similiar questions, matching the query text.

    """  
    #nlp_corpus = [' '.join(x) for x in nlp_preprocessing(corpus)]
    nlp_corpus = [' '.join(x) for x in questions['nlp_tokens']]
    nlp_text = [' '.join(nlp_preprocessing([query_text])[0])]
    vectorizer = TfidfVectorizer(lowercase = True, stop_words = 'english')
    vectorizer.fit(nlp_corpus)
    corpus_tfidf = vectorizer.transform(nlp_corpus)
    
    text_tfidf = vectorizer.transform(nlp_text)
    sim = cosine_similarity(corpus_tfidf, text_tfidf)
    sim_idx = (sim >= threshold).nonzero()[0]
    result = pd.DataFrame({'similarity':sim[sim_idx].reshape(-1,),
                          'text':corpus[sim_idx]},
                          index=sim_idx)
    result = result.sort_values(by=['similarity'], ascending=False).head(top)
    return result

In [ ]:
def get_questions_answers(sim_questions):
    """ Merges the questions with the corresponding answers

    """  
    sim_question_answers = pd.merge(sim_questions, questions, left_index=True, right_index=True)
    sim_question_answers = pd.merge(sim_question_answers, answers, left_on='questions_id', right_on='answers_question_id')
    sim_question_answers = sim_question_answers[['questions_id', 'similarity', 'questions_title', 'questions_body', 'answers_body']]
    return sim_question_answers

In [ ]:
def get_recommendation(sim_questions, plot_graph=True, print_report=True, top_n=5):
    """ Get the top recommended professionals based on questions

    """    
    df = pd.merge(sim_questions, questions, left_index=True, right_index=True)
    df = pd.merge(df, answers, left_on='questions_id', right_on='answers_question_id')
    df = df[['questions_id', 'similarity', 'answers_author_id']]
    plot_data = pd.DataFrame(columns=['source', 'target', 'value'])
    plot_data = plot_data.append(pd.DataFrame({'source':['question'] * len(df['questions_id'].drop_duplicates()),
                                                   'target':df['questions_id'].drop_duplicates(),
                                                  'value':df['similarity'].drop_duplicates()}), ignore_index=True)
    temp_values = df['similarity']/df['questions_id'].apply(lambda x: df.groupby('questions_id').size()[x])
    temp_values = temp_values * df['answers_author_id'].apply(lambda x: df.groupby('answers_author_id').size()[x])
    plot_data = plot_data.append(pd.DataFrame({'source':df['questions_id'],
                                                   'target':df['answers_author_id'],
                                                  'value':temp_values}), ignore_index=True)  
    
    if plot_graph:
        plt.figure(figsize=(15, 15)) 
        G = nx.Graph()
        
        node_color = []
        node_size = []
        G.add_nodes_from(plot_data[plot_data['source'] == 'question']['source'].dropna().unique())
        node_color += (['grey']*len(plot_data[plot_data['source'] == 'question']['source'].dropna().unique()))
        node_size += ([3]*len(plot_data[plot_data['source'] == 'question']['source'].dropna().unique()))
        G.add_nodes_from(plot_data[plot_data['source'] == 'question']['target'].dropna().sort_values().unique())
        node_color += (['blue']*len(plot_data[plot_data['source'] == 'question']['target'].dropna().unique()))
        node_size += (plot_data[plot_data['source'] == 'question']['value'].sort_values().tolist())
        G.add_nodes_from(plot_data[~plot_data['target'].isin(plot_data['source'])]['target'].dropna().sort_values().unique())
        node_color += (['cyan']*len(plot_data[~plot_data['target'].isin(plot_data['source'])]['target'].dropna().unique()))
        node_size += (plot_data[~plot_data['target'].isin(plot_data['source'])].groupby('target')['value'].sum().tolist())
        node_size = [s*800 for s in node_size]

        G.add_edges_from({tuple(row) for i,row in plot_data[['source', 'target']].dropna().iterrows()})
        nx.draw_networkx(G, with_labels=True, node_color=node_color, font_size=8, node_size=node_size)
        plt.title('Recommendation graph from new question to professionals')
        plt.axis('off')
        
        legend_email = mpatches.Patch(color='grey', label='New Question')
        legend_question = mpatches.Patch(color='blue', label='Similar Question')
        legend_student = mpatches.Patch(color='cyan', label='Professional')
        plt.legend(handles=[legend_email, legend_question, legend_student])
        plt.show()

    if print_report:
        top_professionals = plot_data[~plot_data['target'].isin(plot_data['source'])][['target', 'value']].groupby('target').sum()
        top_professionals = top_professionals.reset_index().sort_values('value', ascending = False).head(top_n)
        top_professionals.columns = ['professional', 'recommendation_score']
        print(top_professionals)

## Example 1  <a id="recommender_example1"></a> 
I use a already existing question to get a recommendation. It's a question about the process of becoming a lawyer and how hard it will be.

In [ ]:
sim_corpus = questions['questions_full_text']
sim_text = sim_corpus[seed]
print('Example 1 Question:\n', sim_text)
sim_questions = get_similar_docs(sim_corpus, sim_text, top=10)

**Similar Questions:  **  
In this example the first recommendation is the question itself.  
But a look on the other recommendation seems to be a good match too. They are about *lawyer* and how *hard* it will be.

In [ ]:
sim_questions

**Answers to similar questions**  
Now I can merge the recommended questions with the answers for these questions. This can be used to give the student who asked the question a first recommendation of his question. Maybe these answers are already an answer to his question.

In [ ]:
get_questions_answers(sim_questions).head().T

**Recommended Professionals**  
In the middle we have the <span style="background-color:gray">new question</span> with edges to <span style="background-color:blue">similar questions</span>. The size for the nodes of the similar questions dependence  on the similarity.  
Then we can see what <span style="background-color:blue">similar questions</span> have answered by <span style="background-color:cyan">professionals</span>. The size for the nodes of the professionals dependence on the recommendation score.  
We search a professionals with a big node, because they have a higher recommendation.

The Professional **c5c2ca95fcd3463a8852b8bc9d636313** has the highest score with 2.27. This is because he answered three questions with a high similarity.

In [ ]:
get_recommendation(sim_questions)

## Example 2  <a id="recommender_example2"></a> 
Example 2 use a new defined question about the carrer as a data scientist.

In [ ]:
query_text = 'I will finish my college next year and would like to start a career as a data scientist. \n'\
            +'What is the best way to become a good data scientist? #data-science'
print('Example 2 Question:\n', query_text)
sim_questions = get_similar_docs(sim_corpus, query_text, top=10)

**Similar Questions:  **  
The recommended questions are also about the carrer and preparation of become a data scientist.

In [ ]:
sim_questions

**Answers to similar questions**  
Here we have several answers to the recommended similar questions and can use this to forward the question to a professional.

In [ ]:
get_questions_answers(sim_questions).head(5).T

**Recommended Professionals**  
In this example we have increased the the size to the top 10 similar questions. As we can see there is one professional, who has answered three of this similar questions and seems to be a good recommendation.

In [ ]:
get_recommendation(sim_questions)

# 6. Topic Model (LDA) <a id="lda"></a>  
In this section I will implement a LDA Model to get topic probabilities for the questions. We can use this to see how topics are distributed across questions and which words characterize them.  
New questions can be allocated to topics and forwarded to professional who are familiar with these topics.

1. Use NLP on the Questions corpus.  
    a. Use part-of-speech tagging to filter words.  
    b. Filter extrem values from corpus.  
    c. Calculate the tf-idf. 
2. Train a LDA Model.  
3. Give the topics names.
3. Get the topic probability of a query text.

## Parameters

In [ ]:
# Gensim Dictionary
extremes_no_below = 10
extremes_no_above = 0.6
extremes_keep_n = 8000

# LDA
num_topics = 18
passes = 20
chunksize = 1000
alpha = 1/50

## Functions

In [ ]:
def get_model_results(ldamodel, corpus, dictionary):
    """ Create doc-topic probabilities table and visualization for the LDA model

    """  
    vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)
    transformed = ldamodel.get_document_topics(corpus)
    df = pd.DataFrame.from_records([{v:k for v, k in row} for row in transformed])
    return vis, df  

In [ ]:
def get_model_wordcloud(ldamodel):
    """ Create a Word Cloud for each topic of the LDA model

    """  
    plot_cols = 3
    plot_rows = math.ceil(num_topics / 3)
    axisNum = 0
    plt.figure(figsize=(5*plot_cols, 3*plot_rows))
    for topicID in range(ldamodel.state.get_lambda().shape[0]):
        #gather most relevant terms for the given topic
        topics_terms = ldamodel.state.get_lambda()
        tmpDict = {}
        for i in range(1, len(topics_terms[0])):
            tmpDict[ldamodel.id2word[i]]=topics_terms[topicID,i]

        # draw the wordcloud
        wordcloud = WordCloud( margin=0,max_words=20 ).generate_from_frequencies(tmpDict)
        axisNum += 1
        ax = plt.subplot(plot_rows, plot_cols, axisNum)

        plt.imshow(wordcloud, interpolation='bilinear')
        title = topicID
        plt.title(title)
        plt.axis("off")
        plt.margins(x=0, y=0)
    plt.show()

In [ ]:
def topic_query(data, query):
    """ Get Documents matching the query with the doc-topic probabilities

    """  
    result = data
    result['sort'] = 0
    for topic in query:
        result = result[result[topic] >= query[topic]]
        result['sort'] += result[topic]
    result = result.sort_values(['sort'], ascending=False)
    result = result.drop('sort', axis=1)
    result = result.head(5)
    return result

In [ ]:
def get_text_topics(text, top=20):
    """ Get the topics probabilities for a text and highlight relevant words

    """    
    def token_topic(token):
        return topic_words.get(token, -1)
    
    colors = ['\033[46m', '\033[45m', '\033[44m', '\033[43m', '\033[42m', '\033[41m', '\033[47m']    
    nlp_tokens = nlp_preprocessing([text])

    bow_text = [lda_dic.doc2bow(doc) for doc in nlp_tokens]
    bow_text = lda_tfidf[bow_text]
    topic_text = lda_model.get_document_topics(bow_text)
    topic_text = pd.DataFrame.from_records([{v:k for v, k in row} for row in topic_text])
    

    print('Question:')
    topic_words = []
    topic_labeled = 0
    for topic in topic_text.columns.values:
        topic_terms = lda_model.get_topic_terms(topic, top)
        topic_words = topic_words+[[topic_labeled, lda_dic[pair[0]], pair[1]] for pair in topic_terms]
        topic_labeled += 1
    topic_words = pd.DataFrame(topic_words, columns=['topic', 'word', 'value']).pivot(index='word', columns='topic', values='value').idxmax(axis=1)
    nlp_doc = nlp(text)
    text_highlight = ''.join([x.string if token_topic(x.lemma_.lower()) <0  else colors[token_topic(x.lemma_.lower()) % len(colors)] + x.string + '\033[0m' for x in nlp_doc])
    print(text_highlight) 
    
    print('\nTopics:')
    topic_labeled = 0
    for topic in topic_text:
        print(colors[topic_labeled % len(colors)]+'Topic '+str(topic)+':', '{0:.2%}'.format(topic_text[topic].values[0])+'\033[0m')
        topic_labeled += 1

    # Plot Pie chart
    plt_data = topic_text
    plt_data.columns = ['Topic '+str(c) for c in plt_data.columns]
    plt_data['Others'] = 1-plt_data.sum(axis=1)
    plt_data = plt_data.T
    plt_data.plot(kind='pie', y=0, autopct='%.2f')
    plt.xlabel('')
    plt.ylabel('')
    plt.title('Topics Probabilities')
    plt.show()

## Create Model <a id="lda_model"></a> 

In [ ]:
%%time
lda_tokens = questions['nlp_tokens']
# Gensim Dictionary
lda_dic = gensim.corpora.Dictionary(lda_tokens)
lda_dic.filter_extremes(no_below=extremes_no_below, no_above=extremes_no_above, keep_n=extremes_keep_n)
lda_corpus = [lda_dic.doc2bow(doc) for doc in lda_tokens]

lda_tfidf = gensim.models.TfidfModel(lda_corpus)
lda_corpus = lda_tfidf[lda_corpus]

# Create LDA Model
lda_model = gensim.models.ldamodel.LdaModel(lda_corpus, num_topics=num_topics, 
                                            id2word = lda_dic, passes=passes,
                                            chunksize=chunksize,update_every=0, 
                                            alpha=alpha, random_state=seed)

# Create Visualization and Doc-Topic Probapilities
lda_vis, lda_result = get_model_results(lda_model, lda_corpus, lda_dic)
lda_questions = questions[['questions_id', 'questions_title', 'questions_body']]
lda_questions = pd.concat([lda_questions, lda_result.add_prefix('Topic_')], axis=1)

## Topics  <a id="lda_topics"></a> 
Each wordcloud shows a topic and the top words who define the topic. 
Here some examples:    
Topic 0 is for teacher (*teacher, teaching, education, ...*)  
Topic 1 is for designer (*design, video, graphic, art, ...*)  
Topic 4 is for veterinary (*veterainary, vet, animal, ...*) but seems to be for actors to (*film, theatre, music, singer, ...*)  
Topic 9 is for health (*medicine, doctor, dental, ...*)  
Topic 13 is for engineers (*engineering, mechanical, aerospace, electrical, ...*)  
Topic 17 is for sport (*sport, athlet, basketball, ...*)

In [ ]:
get_model_wordcloud(lda_model)

## Interactive Visualization  
*lda_vis* is a interactive visualization for topic model. But it makes some problems with the sceen width on kaggle, so I commented it out.

In [ ]:
#lda_vis

## Document-Topic Probabilities <a id="lda_doc_topic_prob"></a> 
Here are the topic probabilites for the first five questions.  
Topics with *NaN* values for these five question were deleted.  
If a topic probabilites is under a give threshold it gets automaticaly a *NaN* value

In [ ]:
lda_questions.head(5).dropna(axis=1, how='all').T

## Example 1  <a id="lda_example1"></a> 
The example with the data science text was assigned to topic 3 with 87%.  
The highlighted text are words, who define the topic.  
A look at the previously created wordcloud shows that topic 3 is a mix of *math* and *computer science*.

In [ ]:
query_text = 'I will finish my college next year and would like to start a career as a Data Scientist. \n\n'\
            +'What is the best way to become a good Data Scientist? #data-science'
get_text_topics(query_text, 80)

## Example 2  <a id="lda_example2"></a>
Now I would like to make a query, which gives me back documents with the topic *veterinary (**Topic 4**)* and *health (**Topic 9**)*.  
The first two questions are about descision to begin the career as a veterinarian (*Topic 4*) or in another medical field (*Topic 9*).

In [ ]:
query = {'Topic_4':0.4, 'Topic_9':0.4}
topic_query(lda_questions, query).dropna(axis=1, how='all').head(2).T

In [ ]:
get_text_topics(questions['questions_full_text'][20658], 50)
print()
get_text_topics(questions['questions_full_text'][3075], 50)